In [40]:
pip install 

Note: you may need to restart the kernel to use updated packages.


ERROR: You must give at least one requirement to install (see "pip help install")

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [41]:
import pymongo
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import re
import time
from datetime import date
from datetime import timedelta
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime 
import json
import requests
from pymongo.errors import BulkWriteError
from pprint import pprint
import nltk
import string
nltk.download('stopwords')
from nltk.corpus import stopwords
import emoji
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from itertools import chain
from collections import Counter
import joblib 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_curve
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from joblib import dump, load
pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abdul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [42]:
# client = pymongo.MongoClient("mongodb://localhost:27017")
# db = client['Final_Year_Project']
# collection = db['stock_twits_dataset']
# records = collection.find({})   # Getting all the dociments from mongodb
# # print(type(records))  # checking the type

In [43]:
# list_records = [record for record in records]
# list_records = list(records) 
# df = pd.DataFrame(list_records)
# # df.head()

In [44]:
# for i in records:
#     p = 1
#     print(i)
#     p = p + 1
#     if p == 10 :
#         break

In [45]:
## DATA FOR PREDICTION (it is yesterday data of the current data)
stock_name = 'EASEMYTRIP.NSE'
header = {'Accept' : 'application/json',
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36',
    'Accept-Encoding':'gzip, deflate, br',
    'Accept-Language' :  'en-IN,en-GB;q=0.9,en-US;q=0.8,en;q=0.7,hi;q=0.6',
    'Connection': 'keep-alive',
    'Sec-Fetch-Site':'same-site',
    'Sec-Fetch-Mode':'cors',
    'Sec-Fetch-Dest':'empty'}


              # this name will come from flask frontend
main_data = []                    #  All data will store here
api_next_list = []


time = 24                             # Change according to number of days
current_time = datetime.now()                 # It show current time
time = current_time - timedelta(hours=time)   # it will go 24 hours back and give you a specific time and time
# time = 24
# date_of_testing ='12/15/23'
# date_of_testing_datetime = datetime.strptime(date_of_testing, '%m/%d/%y')
# time = date_of_testing_datetime - timedelta(hours=time)

#Initializing the request and converts it into json
url = f'https://api.stocktwits.com/api/2/streams/symbol/{stock_name}.json?filter=top&limit=22'  
request_success = requests.get(url,headers=header)   
json_data_dict = request_success.json()


#Taking all the usefull data from the json data
usefull_data = (json_data_dict['messages']) 

#picking the next api address for further scrapping
next_api_first = json_data_dict['cursor']['max']
next_api_first = str(next_api_first)
next_api_first = '&max='+next_api_first



# Retrieving the last date from twit to break the first loop
last_twit_date_1 = (json_data_dict['messages'][-1])
last_twit_date_1 = last_twit_date_1['created_at']
last_twit_date_1 = re.sub(pattern = "[T-Z]+",repl = ' ',string=last_twit_date_1) # eliminating alphabets
last_twit_date_1 = datetime.strptime(last_twit_date_1, "%Y-%m-%d  %H:%M:%S ")  # changing datatype



#Looping in the usefull_data
for data in usefull_data:
    # Getting the twit date from  usefull_data and cleaning is done
    twit_date = data['created_at']
    twit_date = re.sub(pattern = "[T-Z]+",repl = ' ',string=twit_date)
    twit_date = datetime.strptime(twit_date, "%Y-%m-%d  %H:%M:%S ")   

    if twit_date > time:
        twit = data['body']
        signal = data['entities']['sentiment']
        if signal == None:
            signal = 'Null'
        else :
            signal = signal['basic']
        stock_twits_data = {'stock_name':stock_name ,'Date':twit_date,'Twits': twit,'Signal': signal,'Cleaning_status':'pending','Cleaning_status_lemmatization':'pending'}
        main_data.append(stock_twits_data)
    if twit_date < time:
        if len(main_data) == 0:
            print('No twits found')
        else:
            print('twits are there')
        break
    try:
        if time < last_twit_date_1.strptime('%H:%M:%S'):
            break
    except:
        pass

#########################################################################################################################################    
while (True):
    loop_break = []
    url =  f'https://api.stocktwits.com/api/2/streams/symbol/{stock_name}.json?filter=top&limit=22{next_api_first}'
    request_success = requests.get(url,headers=header)   
    json_data_dict = request_success.json() 
    next_api_first = json_data_dict['cursor']['max']
    next_api_first = str(next_api_first)
    next_api_first = '&max='+next_api_first
    usefull_data = (json_data_dict['messages'])
    for data in usefull_data:
        twit_date = data['created_at']
        twit_date = re.sub(pattern = "[T-Z]+",repl = ' ',string=twit_date)
        twit_date = datetime.strptime(twit_date, "%Y-%m-%d  %H:%M:%S ")   
        if twit_date > time:
            twit = data['body']
            signal = data['entities']['sentiment']
            if signal == None:
                signal = 'Null'
            else :
                signal = signal['basic']
            stock_twits_data = {'stock_name':stock_name ,'Date':twit_date,'Twits': twit,'Signal':signal,'Cleaning_status':'pending','Cleaning_status_lemmatization':'pending'}
            main_data.append(stock_twits_data)



    if twit_date < time:
        print('while loop break')
        break
print(main_data)
if not main_data:    
    print('No data is found')
else :
    df = pd.DataFrame(main_data)
    pd.set_option('display.max_colwidth', None)


    ##Removing Links from dataframe
    print('---------------------------------Links------------------------------------')
    df['Twits'] = df['Twits'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)


    ## Lowercase the whole twits column
    print('---------------------------------lowercase------------------------------------')
    df['Twits'] = df['Twits'].apply(lambda x: x.lower())


    ## Eliminating the pumctuation 
    print('----------------------------eliminate punctuation-------------------------------')
    df['Twits'] = df['Twits'].str.replace('[^\w\s]',' ',regex=True)


    ##Eliminating stopwords 
    def remove_stopwords(x):
        return " ".join([word for word in str(x).split() if word not in stop_words])
    stop_words = set(stopwords.words('english'))
    print('----------------------------removing stopwords-------------------------------')
    df['Twits'] = df['Twits'].apply(lambda x : remove_stopwords(x))


    ##Eliminating Emoji from dataframe
    print('----------------------------removing emojis-------------------------------')
    df['Twits'] = df['Twits'].apply(lambda s: emoji.replace_emoji(s, ''))

    # Stemming on dataframe
    print('----------------------------stemming is done-------------------------------')
    stemmer = PorterStemmer()
    df['Twits'] = df['Twits'].str.split()
    df['Twits'] = df['Twits'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.
    df['Twits'] = df['Twits'].apply(lambda x: " ".join(x))

    # Removing number str from column twits
    print('----------------------------removing number is done-------------------------------')
    df['Twits'] = df['Twits'].replace('\d+', '', regex=True)

    # Removing the words which contain only 1 letter
    print('----------------------------removing 1 letter word is done-------------------------------')
    df['Twits'] = df['Twits'].str.split()
    for text in df['Twits']:
        for word in text:
            if len(word) == 1:
                text.remove(word)
    df['Twits'] = df['Twits'].apply(lambda x: " ".join(x))
    df_yesterday = df.drop(['stock_name','Date','Cleaning_status','Cleaning_status_lemmatization','Signal'], axis=1)
    df_yesterday = df_yesterday.drop_duplicates()
    print(df_yesterday)

In [46]:
# df_yesterday

# ------------------------------------------------------------------------------------------------------------------------

In [48]:
df = pd.read_csv('cleaned_data.csv')

In [49]:
df = df.drop(['_id', 'index','stock_name','Date'], axis=1)

In [ ]:
# find duplicate value 
# duplicate = df[df.duplicated()]

#drop all duplicate rows from dataframe (df)
df = df.drop_duplicates()
df

In [ ]:
# df[df['twits']=='tatasteel nse'].drop_duplicates()  # see all the tatasteel.nse rows from here
df = df.drop([0, 8 ,492])   # it will remove the rows that contains tatasteel.nse 
df

In [ ]:
# --> take all the null signal value for testing value      ##### Testing data named----> df_testing

df_null = df[df['signal'] == 'Null']
df_testing = df_null
df_testing_x_test = df_testing['twits']
df_testing_x_test = df_testing_x_test.astype('unicode')
df_testing_y_test = df_testing['signal']
# df_testing['twits'] = df_testing['twits'].astype('unicode')


In [ ]:
df

In [ ]:
df_testing_y_test

In [ ]:
# # --> take all the bearish signals value      ##### training data

df_bearish = df[df['signal'] == 'Bearish']
df_bearish

In [ ]:
# # --> take all the bullish signals value      ##### training data

df_bullish = df[df['signal'] == 'Bullish']
df_bullish = df_bullish.reset_index(drop=True)   # reset index to count the numbers properly
df_bullish = df_bullish.iloc[0:665007]           # iloc is used to equalize the Bearish and Bullish counts
df_bullish

In [ ]:
# # --> merging two dataframe for training dataset

dataframes = [df_bullish,df_bearish]
df = pd.concat(dataframes)

In [ ]:
df

# The Above Implemantation is different and the below implementation is different

In [ ]:
df = pd.read_csv('cleaned_data.csv')

In [ ]:
# --> take all the null signal value for testing value      ##### Testing data named----> df_testing


# df_null = df[df['signal'] == 'Null']
# df_testing = df_null
# df_testing  # How many null values are there

In [ ]:
# -->Only get Bullisg and Bearish value only eliminating null value    ##### Training data named-----> df


# df.drop(df[(df['signal']=='Null')].index, inplace=True)

# The Below code is common for both methods

In [ ]:
# creating a new column for target value named == 'signal for machine'

df['signal for machine'] = df['signal'].map({'Bullish': 1,'Bearish': 0})

In [ ]:
df.head(50)

In [ ]:
# Check the target value the number of count of bullish and bearish (1,0)
df['signal for machine'].value_counts().plot(kind='bar')

In [ ]:
# --> First convert the Object dtype to unicode and then go to train test split


df['twits'] = df['twits'].astype('unicode')

In [ ]:
# Train Test Split


X_train, X_test, y_train, y_test = train_test_split(df['twits'], df['signal for machine'], test_size=0.1,random_state=42)

In [ ]:
print(X_train)

In [ ]:
print(y_train)

In [ ]:
X_test

In [ ]:
print(y_test)

In [ ]:
#TF-IDF Implementation now Training data and testing data
vect = TfidfVectorizer()

In [ ]:
# Fitting the tfidf vectorizer and saving it as a file
X_train_vectorized = vect.fit(X_train)
print(type(X_train_vectorized))
joblib.dump(X_train_vectorized, 'tfidf_vectorizer_2.pkl')

In [ ]:
# Loading a tfidf vectorizer and transforming X_train and X_test

loaded_tfidf_vectorizer = joblib.load('tfidf_vectorizer_2.pkl')
X_train_vectorized = loaded_tfidf_vectorizer.transform(X_train)
print(type(X_train_vectorized))
X_test_vectorized  = loaded_tfidf_vectorizer.transform(X_test)
print(type(X_test_vectorized))

In [ ]:
# print(type(X_test))

In [ ]:
# # This converts the words into decimals so the model can understand    testing data
# X_test_vectorized = vect.transform(X_test)
# pd.DataFrame(X_test_vectorized)
# print("Shape of X_test_tfidf:", X_test.shape)

In [ ]:
# This converts the words into decimals so the model can understand    dataset wants to predicts (Null values dataset)
loaded_tfidf_vectorizer = joblib.load('tfidf_vectorizer_2.pkl')
x_test_prediction = loaded_tfidf_vectorizer.transform(df_testing_x_test)
print("Shape of X_test_null_value_data:", x_test_prediction.shape)


In [ ]:
df_yesterday

In [ ]:
# # This converts the words into decimals so the model can understand    dataset wants to predict (yesterday dataset)
# df_yesterday = df_yesterday['Twits'].astype('unicode')
# tfidf_vectorizer_prediction_yesterday = TfidfVectorizer(max_features=1000)
# x_test_prediction_yesterday = tfidf_vectorizer_prediction_yesterday.fit_transform(df_yesterday)
# print("Shape of yesterday_dataset:", x_test_prediction_yesterday.shape)

In [ ]:
pd.DataFrame(x_test_prediction_yesterday)

In [ ]:
pd.DataFrame(x_test_prediction_yesterday)

In [ ]:
pd.DataFrame(df_yesterday)

In [ ]:
### Making prediction on a single twits Merging all the rows of dataframe   IMPO
# Merging all therows of dataframe in a list  
pd.DataFrame(df_yesterday)
df_yesterday_combine         = df_yesterday                                            # duplicating the df_yesterday
twits_prediction_lists       = df_yesterday_combine['Twits'].tolist()                                   # Converting it into list
twits_prediction_lists_merge = " ".join(twits_prediction_lists)                        # joining each element of a list into a single element
dict_yesterday_twits_merge   = {'Twits':twits_prediction_lists_merge}                  # Initializing of creating dataframe  
df_yesterday_twits_merge     = pd.DataFrame(dict_yesterday_twits_merge,index=['Index'])# making dataframe
df_yesterday_twits_merge_unicode  = df_yesterday_twits_merge['Twits'].astype('unicode')# Converting it into unicode
loaded_tfidf_vectorizer = load('tfidf_vectorizer.pkl')
x_test_prediction_yesterday_merge = loaded_tfidf_vectorizer.transform(df_yesterday_twits_merge_unicode) 
print("Shape of yesterday_dataset:", x_test_prediction_yesterday_merge.shape)          #Checking the shape  

# Linear Regression   ( 78 Accuracy )

In [ ]:
## creating Model using Logistic regression   X_train_vectorized twits(decimals tfidf)     y_train contain (1 and 0)  target data
model = LinearRegression()
model.fit(X_train_vectorized,y_train)  # actucal training of data will we here



## Can give any test data but first convert it into object into unicode     X_test twits (decimals tfidf)    # we can give any dataset here
## converting into decimals with tfidf
# predictions = model.predict(vect.transform(df_testing_x_test))
predictions = model.predict(X_test_vectorized)     # Give here df_testing


# It is in Numbers means 0 and 1
print(predictions)  # the preduicted values is in 1 == Bullish and 0 == Bearish


# How to convert numbers into str 
predictions_in_str = []
print(predictions_in_str)
for i in predictions:
    i = round(i) # getting approximate value by using round function 
    i = str(i)  # First we are converting it into string 
    predictions_in_str.append(i)  # appending in a empty list
    

predictions_in_str = list(map(lambda x: x.replace('1', 'Bullish'), predictions_in_str))   # Converting 1 into Bullish 
predictions_in_str = list(map(lambda x: x.replace('0', 'Bearish'), predictions_in_str))   # Converting 0 into Bearish


# # if you want to see the changes in the predictions_in_str 
# for i in predictions_in_str:
# #     print(i)
#     z = 20
#     q = 1
#     q = q +1
#     if q == 20:
#         break
        
# compute AUC value--> Area Under Curve
# print("AUC:",roc_auc_score(y_test,predictions))


# accuracy value of linear regression
predictions_list_converts_again = []
for i in predictions:
    i = round(i)
    predictions_list_converts_again.append(i)
print(accuracy_score(y_test,predictions_list_converts_again))
# model_train_accuracy = accuracy_score(y_train,y_train_pred)

In [ ]:
# Here we Combine the X_test and the predicted value
print(len(predictions_in_str))
df_result_final = pd.DataFrame(X_test)
df_result_final['Predicted value final'] = predictions_in_str
df_result_final = df_result_final.sort_index(axis=0)  # sort in ascending order

df_result_final.head(50)

In [ ]:
# Saving the model and loading it
dump(model, 'linear_regression_final.joblib')
# loaded_model = load('linear_regression.joblib')

In [ ]:
# Making predictions on NULL dataset
loaded_model = load('linear_regression_final.joblib')
predictions = loaded_model.predict(x_test_prediction)  # now see the yesterday dataseton this line
print(predictions)
pd.DataFrame(df_testing_x_test)
# df_testing_x_test.shape

In [ ]:
#Making prediction on a single merge twit yesterday data
loaded_model = load('linear_regression_final.joblib')
predictions = loaded_model.predict(x_test_prediction_yesterday_merge)
print(f'The predicted value is {predictions}')
if predictions > 0.5 :
    print('Bullish')
else:
    print('Bearish')


In [ ]:
# Making predictions on yesterday dataset
loaded_model = load('linear_regression.joblib')
predictions = loaded_model.predict(x_test_prediction_yesterday)  # now see the yesterday dataseton this line
results_final = []
for signal in predictions:
    if signal >= 0.5:
        results_final.append("Bullish")
    else:
        results_final.append("Bearish") 
print(len(predictions))

# Combining the twits and predictions of yesterday data tranforming it into dataframe
df_yesterday_prediction = pd.DataFrame(df_yesterday)                     # creating a dataframe of yesterday twits
df_yesterday_prediction['Yesterday_results_prediction'] = results_final  # adding the prediction value in new column 
df_yesterday_prediction

In [ ]:
# Final decision of Bullisg and Bearish 
bullish = []
bearish = []
for i in results_final:
    if i == 'Bullish':
        bullish.append(i)
    else:
        bearish.append(i)
if len(bullish) > len(bearish):
    print('Stock is Bullish')
else:
    print('Stock is Bearish')

In [ ]:
twits_prediction_lists_merge

In [ ]:

point_five_greater_bullish = []
point_five_lesser_bearish  = []
for pred in predictions:
    if pred > 0.5 :
        point_five_greater_bullish.append(pred)
    else:
        point_five_lesser_bearish.append(pred)
bullish_list_numbers = sum(point_five_greater_bullish)
bearish_list_numbers = sum(point_five_lesser_bearish)
print(bullish_list_numbers)
print(bearish_list_numbers)
if bullish_list_numbers > bearish_list_numbers:
    print('Stock is Bullish')
else :
    print('Stock is Bearish')

In [ ]:
predictions

In [ ]:
print(len(bullish))

In [ ]:
print(len(bearish))

In [ ]:
predictions

# Logistic regression   (79 Accuracy)

In [ ]:
## creating Model using Logistic regression   X_train_vectorized twits(decimals tfidf)     y_train contain (1 and 0)  target data
model = LogisticRegression()
model.fit(X_train_vectorized,y_train)



## Can give any test data but first convert it into object into unicode     X_test twits (decimals tfidf)    # we can give any dataset here
## converting into decimals with ifidf
predictions = model.predict(X_test_vectorized)    # Give here df_testing


# It is in Numbers means 0 and 1
print(predictions)  # the preduicted values is in 1 == Bullish and 0 == Bearish


# How to convert numbers into str 
predictions_in_str = []
for i in predictions:
    i = str(i)  # First we are converting it into string 
    predictions_in_str.append(i)  # appending in a empty list
predictions_in_str = list(map(lambda x: x.replace('1', 'Bullish'), predictions_in_str))   # Converting 1 into Bullish 
predictions_in_str = list(map(lambda x: x.replace('0', 'Bearish'), predictions_in_str))   # Converting 0 into Bearish


# # if you want to see the changes in the predictions_in_str 
# for i in predictions_in_str:
# #     print(i)
#     z = 20
#     q = 1
#     q = q +1
#     if q == 20:
#         break
        
# compute AUC value--> Accuracy
predictions_list_converts_again = []
for i in predictions:
    i = round(i)
    predictions_list_converts_again.append(i)
print(accuracy_score(y_test,predictions_list_converts_again))

In [ ]:
# Here we Combine the X_test and the predicted value
print(len(predictions_in_str))
df_result_final = pd.DataFrame(X_test)
df_result_final['Predicted value final'] = predictions_in_str
df_result_final = df_result_final.sort_index(axis=0)  # sort in ascending order

df_result_final.head(50)

In [ ]:
# Saving the model and loading it
dump(model, 'logistic_regression_final.joblib')
# loaded_model = load('linear_regression.joblib')

In [ ]:
# Making predictions on NULL dataset
loaded_model = load('linear_regression_final.joblib')
predictions = loaded_model.predict(x_test_prediction)  # now see the yesterday dataseton this line
print(predictions)
pd.DataFrame(df_testing_x_test)
# df_testing_x_test.shape

In [ ]:
#Making prediction on a single merge twit yesterday data
loaded_model = load('logistic_regression_final.joblib')
predictions = loaded_model.predict(x_test_prediction_yesterday_merge)
print(f'The predicted value is {predictions}')
if predictions > 0.5 :
    print('Bullish')
else:
    print('Bearish')

In [ ]:
# Making predictions on yesterday dataset
loaded_model = load('logistic_regression.joblib')
predictions = loaded_model.predict(x_test_prediction_yesterday)  # now see the yesterday dataseton this line
results_final = []
for signal in predictions:
    if signal >= 0.5:
        results_final.append("Bullish")
    else:
        results_final.append("Bearish") 
print(len(predictions))

# Combining the twits and predictions of yesterday data tranforming it into dataframe
df_yesterday_prediction = pd.DataFrame(df_yesterday)                     # creating a dataframe of yesterday twits
df_yesterday_prediction['Yesterday_results_prediction'] = results_final  # adding the prediction value in new column 
df_yesterday_prediction

In [ ]:
# Final decision of Bullisg and Bearish 
bullish = []
bearish = []
for i in results_final:
    if i == 'Bullish':
        bullish.append(i)
    else:
        bearish.append(i)
if len(bullish) > len(bearish):
    print('Stock is Bullish')
else:
    print('Stock is Bearish')

In [ ]:
point_five_greater_bullish = []
point_five_lesser_bearish  = []
for pred in predictions:
    if pred > 0.5 :
        point_five_greater_bullish.append(pred)
    else:
        point_five_lesser_bearish.append(pred)
bullish_list_numbers = sum(point_five_greater_bullish)
bearish_list_numbers = sum(point_five_lesser_bearish)
print(bullish_list_numbers)
print(bearish_list_numbers)
if bullish_list_numbers > bearish_list_numbers:
    print('Stock is Bullish')
else :
    print('Stock is Bearish')

In [ ]:
# Making predictions on yesterday dataset

In [ ]:
# def GiveYourDataset(dataset):
#     dataset = dataset.astype('unicode')   # converting dataset to unicodes
#     print('step 01 done : converting dataset to unicodes ')
    
#     tfidf_vectorizer_prediction_ = TfidfVectorizer(max_features=10000)  # creating tfidf
#     print('step 02 done : creating tfidf')
    
#     x_test_prediction = tfidf_vectorizer_prediction_.fit_transform(dataset) # fitting and transform tfidf
#     print('step 03 done: fitting and transform tfidf')
    
#     shape_of_prediction = x_test_prediction.shape
#     loaded_model = load('linear_regression.joblib')  # loading the model
#     print('step 04 done : loading the model')
    
#     predictions_final = loaded_model.predict(x_test_prediction)  # making predictions
#     print('step 05 done : Prediction done all set')
    
#     return predictions_final, shape_of_prediction

In [ ]:
# GiveYourDataset(df_yesterday)

In [ ]:
# # appending in a list the repected columns value i.e signal

# actual_value = y_test.tolist()    # here we are converting y_test which is series into list

# # How to convert numbers into str 
# actual_value_in_str = []
# for i in actual_value:
#     i = str(i)  # First we are converting it into string 
#     actual_value_in_str.append(i)  # appending in a empty list
    

# actual_value_in_str = list(map(lambda x: x.replace('1', 'Bullish'), actual_value_in_str))   # Converting 1 into Bullish 
# actual_value_in_str = list(map(lambda x: x.replace('0', 'Bearish'), actual_value_in_str))   # Converting 0 into Bearish


# # Making one column from datframe into list

# predicted_value = df_result['Predicted value'].to_list()

# predicted_value

In [ ]:
# p = 0
# for i in predicted_value:
#     p = p + 1
#     print(i)
#     if p > 10:
#         break

In [ ]:
# print(len(actual_value_in_str))

In [ ]:
# print(len(predicted_value))

In [ ]:
# --> entry that is having 'tatasteel nse' solv it later

# df[df['twits']=='tatasteel nse']

# Calculating Confusion matrix  --> When your data is unbalanced

In [ ]:
#Training dataset(Which is ground truth) and predicted dataset(Which dataset you predicted)
# cm = confusion_matrix(actual_value_in_str,predicted_value)

# For ideal confusion matrix it will show perfection
# cm2= confusion_matrix(actual_value_in_str,actual_value_in_str)

In [ ]:
# print(cm2)   # this we want 

In [ ]:
# print(cm)  # We are gettinmg this

In [ ]:
# from sklearn.metrics import fbeta_score, make_scorer
# ftwo_scorer = make_scorer(fbeta_score, beta=2, average='weighted')
# scorer = sklearn.metrics.make_scorer(sklearn.metrics.f1_score, average = 'weighted')
# gs_svc = GridSearchCV(estimator=svc_clf,param_grid=param_grid,scoring=scorer,cv=5)
# scoring = {'accuracy': make_scorer(accuracy_score),
#            'precision': make_scorer(precision_score, average = 'macro'),
#            'recall': make_scorer(recall_score, average = 'macro'),
#            'f1_macro': make_scorer(f1_score, average = 'macro'),
#            'f1_weighted': make_scorer(f1_score, average = 'weighted')}

In [ ]:
# # Calculating precision

# precision_score(actual_value_in_str,predicted_value, pos_label="Bullish")  # pos_label --> which one is positive 

In [ ]:
# Calculating recall
# recall_score(actual_value_in_str,predicted_value, pos_label="Bullish")

In [ ]:
# Calculating F1-score
# f1_score(actual_value_in_str,predicted_value, pos_label="Bullish")

# Redge and lasso - If overfitting (model working good with train data and working bad with testing data)

In [ ]:
# precision, recall, threshold = precision_recall_curve(actual_value_in_str,predicted_value, pos_label="Bullish")

In [ ]:
# df = df.drop(['signal for machine'], axis=1)
# df = df.set_index('twits',inplace=True)


In [ ]:
# df.reset_index(drop=True, inplace=True)

In [ ]:
# df = df.reset_index(drop=True)

In [ ]:
# df_result = df_result.reset_index(drop=True)

In [ ]:
# df_result

In [ ]:
# display(df)

In [ ]:
# df_result

In [ ]:
# X_train

In [ ]:
# df.index()